In [43]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import openai


In [58]:
class WebscraperSummarizer:
    url: str
    title:str
    content:str
    system_prompt: str
    user_prompt: str
    context: list()
    summarized_content: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'html.parser')

        self.title = soup.title.string if soup.title else 'no title found'
        
        #remove tags
        for tags in soup.body(['script','style','img','input']):
            tags.decompose()
        
        self.content = soup.body.get_text(separator='\n', strip=True)
        load_dotenv()
        #os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
        openai.api_key = os.getenv('OPENAI_API_KEY')
        
        self.system_prompt = """You are a reliable assistant that analyze website content. 
                                You provide a good summary and navigation links and images. 
                                You translate technical words to a very understable way.
                                You always respond in markdown."""

        self.user_prompt = f"You are reading  this website {self.title}"
        self.user_prompt += "The contents of this website is as follows;" + self.content 
        self.user_prompt += "Provide a short summary of this website. Summarize news or announcement.\n\n"
        self.user_prompt += "The final summary should be in structured as markdown.\n\n"


    def messages(self):
        self.context = [
             {'role':'system', 'content': self.system_prompt},
             {'role':'user',   'content': self.user_prompt},
         ]
       
        
        return self

    def summarizer(self):
        
        response=openai.chat.completions.create(
            model='gpt-4o-mini',
            messages=self.context)
        
        self.summarized_content=response.choices[0].message.content
        return self

    def view(self):
        display(Markdown(self.summarized_content))
        return self
    


In [ ]:
url = 'https://edition.cnn.com/'
(
                WebscraperSummarizer(url)
                        .messages()
                        .summarizer()
                        .view()
)
